In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Sudoku

The sudoku we want to solve is shown below:
    <img src="sudoku.png">
It can be represented as a list of lists.

In [ ]:
def create_puzzle():
    return [ ["*",  3 ,  9 , "*", "*", "*", "*", "*",  7 ], 
             ["*", "*", "*",  7 , "*", "*",  4 ,  9 ,  2 ],
             ["*", "*", "*", "*",  6 ,  5 , "*",  8 ,  3 ],
             ["*", "*", "*",  6 , "*",  3 ,  2 ,  7 , "*"],
             ["*", "*", "*", "*",  4 , "*",  8 , "*", "*"],
             [ 5 ,  6 , "*", "*", "*", "*", "*", "*", "*"],
             ["*", "*",  5 ,  2 , "*",  9 , "*", "*",  1 ],
             ["*",  2 ,  1 , "*", "*", "*", "*",  4 , "*"],
             [ 7 , "*", "*", "*", "*", "*",  5 , "*", "*"]
           ]

The function `allDifferent` takes a list `L` as its argument.  This list contains both variables and numbers.
It returns a set of formulas expressing that all variables from `L` take different values.

In [ ]:
def allDifferent(Variables):
    return { f'{x} != {y}' for x in Variables 
                           for y in Variables
                           if  x < y
           }

In [ ]:
def constraints_from_puzzle(Puzzle):
    return { f'V{row+1}{col+1} == {Puzzle[row][col]}' for row in range(9)
                                                      for col in range(9)
                                                      if  Puzzle[row][col] != '*'
           }

In [ ]:
constraints_from_puzzle(create_puzzle())

The function `sudoku_csp` returns a CSP that encodes the given sudoku as a CSP.

In [ ]:
def sudoku_csp():
    Puzzle = create_puzzle()
    L           = range(1, 9+1)
    Variables   = { f'V{row}{col}' for row in L
                                   for col in L
                  }
    Values      = set(L)
    Constraints = constraints_from_puzzle(Puzzle)
    # all entries in a row are unique
    for row in L:
        Constraints |= allDifferent({ f'V{row}{col}' for col in L })
    # all entries in a column are unique
    for col in L:
        Constraints |= allDifferent({ f'V{row}{col}' for row in L })
    # all entries in a square are unique    
    for r in range(3):
        for c in range(3):
            S = { f'V{r*3+row}{c*3+col}' for row in range(1, 3+1)
                                         for col in range(1, 3+1) 
                }
            Constraints |= allDifferent(S)
    return Variables, Values, Constraints

In [ ]:
sudoku_csp()

Given a sudoku `Puzzle` and a `Solution` for this puzzle, the function `find_alternative` computes a CSP
that is solvable iff the puzzle has a second solution that is different from `Solution`.

In [ ]:
def find_alternative(csp, Solution):
    Variables, Values, Constraints = csp
    formula = ' or '.join({ f'{var} != {Solution[var]}' for var in Variables })
    return Variables, Values, Constraints | { formula }

## Graphical Representation

The following line needs to be executed once to install the package `problem_visuals`.

In [ ]:
!pip install git+https://github.com/reclinarka/problem_visuals

In [ ]:
from problem_visuals.games.sudoku.grid import Grid

The function `show_solution` prints the solution.

In [ ]:
def show_solution(Solution, width='50%'):
    Sudoku = create_puzzle()
    Assignment = Solution.copy()
    for row in range(9):
        for col in range(9):
            if Sudoku[row][col] != '*':
                del Assignment[f'V{row+1}{col+1}']
    return Grid(state=Sudoku, assigned=Assignment, html_width=width)